In [44]:
from selenium import webdriver
import pandas as pd
import os
import datetime
from multiprocessing.pool import ThreadPool
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import glob
from bs4 import BeautifulSoup
import shutil
chromeOptions = Options()
from time import sleep
import json
import re

In [7]:
driver = webdriver.Chrome(
    ChromeDriverManager().install(),
    )

[WDM] - Downloading: 100%|██████████| 8.04M/8.04M [00:00<00:00, 10.2MB/s]


In [21]:
url = "https://economictimes.indiatimes.com/markets/stocks/recos"
driver.get(url)

In [8]:
driver.close()

In [65]:
auto_load = driver.find_element(By.CLASS_NAME, "autoload_continue")
driver.execute_script("arguments[0].scrollIntoView();", auto_load)
driver.execute_script("arguments[0].click();", auto_load)

# auto_load.click()

In [106]:
stories = driver.find_elements(By.CLASS_NAME, "eachStory")

In [107]:
story = stories[0]

In [108]:
link = story.find_element(By.TAG_NAME, "a").get_attribute("href")
title = story.find_element(By.TAG_NAME, "a")
time = story.find_element(By.TAG_NAME, "time")
p = story.find_element(By.TAG_NAME, "p").text
print(link, title, time, p)

https://economictimes.indiatimes.com/markets/stocks/recos/buy-persistent-systems-target-price-rs-5880-hdfc-securities/articleshow/99784037.cms <selenium.webdriver.remote.webelement.WebElement (session="ebe24b6789bb1c8f26318206895c4e3e", element="bf1800a6-7def-4c4d-b8bd-2081a4e65829")> <selenium.webdriver.remote.webelement.WebElement (session="ebe24b6789bb1c8f26318206895c4e3e", element="ff523c9c-5b15-4e5b-aa00-c9f6c714b333")> Persistent Systems, incorporated in the year 1990, is a Mid Cap company (having a market cap of Rs 34167.32 Crore) operating in IT Software sector.


In [77]:
html = story.get_property("innerHTML")

In [78]:
soup = BeautifulSoup(html, "html.parser")

In [93]:
tag = str(soup.find("time"))
tag

'<time class="date-format" data-time="Apr 26, 2023, 02:31 PM IST">12 Hours ago</time>'

In [97]:
tag.split('"')[3]

'Apr 26, 2023, 02:31 PM IST'

In [12]:
def get_stories(n = 10):
    driver = webdriver.Chrome(
    ChromeDriverManager().install(),
    )
    url = "https://economictimes.indiatimes.com/markets/stocks/recos"
    driver.get(url)
    sleep(10)
    
    for i in range(n):
        # print(i)
        try:
            close = driver.find_element(By.CLASS_NAME, "imgClose")
            close.click()
        except:
            pass
        sleep(2)
        auto_load = driver.find_element(By.CLASS_NAME, "autoload_continue")
        driver.execute_script("arguments[0].scrollIntoView();", auto_load)
        driver.execute_script("arguments[0].click();", auto_load)
    # sleep(200)
    res_stories = {
        "title":[],
        "link":[],
        "time":[],
        "text":[]
    }
    
    stories = driver.find_elements(By.CLASS_NAME, "eachStory")
    for story in stories:
        link = story.find_element(By.TAG_NAME, "a").get_attribute("href")
        title = story.find_element(By.TAG_NAME, "a").text
        time = story.find_element(By.TAG_NAME, "time")
        p = story.find_element(By.TAG_NAME, "p").text
        html = story.get_property("innerHTML")
        soup = BeautifulSoup(html, "html.parser")
        time_tag = str(soup.find("time"))
        time = time_tag.split('"')[3]
        res_stories["title"].append(title)
        res_stories["link"].append(link)
        res_stories["time"].append(time)
        res_stories["text"].append(p)
    driver.close()
    return pd.DataFrame(res_stories)


In [13]:
df = get_stories(50)

In [14]:
df.tail()

,title,link,time,text
843,"Buy NTPC, target price Rs 169: ICICI Direct",https://economictimes.indiatimes.com/markets/s...,"Feb 14, 2023, 09:41 AM IST","NTPC, incorporated in the year 1975, is a Larg..."
844,"Buy Housing Development Finance Corporation, t...",https://economictimes.indiatimes.com/markets/s...,"Feb 14, 2023, 09:35 AM IST",Price resuming uptrend after consolidating abo...
845,"Sell LIC Housing Finance, target price Rs 356:...",https://economictimes.indiatimes.com/markets/s...,"Feb 14, 2023, 09:28 AM IST","LIC Housing Finance, incorporated in the year ..."
846,"Buy NTPC, target price Rs 176: ICICI Securities",https://economictimes.indiatimes.com/markets/s...,"Feb 14, 2023, 09:23 AM IST","Shallow retracement, higher base at 100 days E..."
847,"Buy Larsen & Toubro, target price Rs 2290: ICI...",https://economictimes.indiatimes.com/markets/s...,"Feb 14, 2023, 09:17 AM IST","Larsen & Toubro, incorporated in the year 1946..."


In [90]:
df.to_csv("data/news.csv", index=None)

In [94]:
df = pd.read_csv("data/news.csv")

In [100]:
df = df.dropna(subset=["title"])

In [101]:
df.head(40)

,title,link,time,text,buy/sell,fund,target_price,stock
0,"Buy Voltas, target price Rs 1005: BNP Paribas ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 30.3 per cent stake in the comp...,Buy,BNP Paribas Securities,1005.0,Voltas
1,"Buy UTI Asset Management Company, target price...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,The brokerage expects the company to deliver 6...,Buy,HDFC Securities,770.0,UTI Asset Management Company
2,"Sell Shoppers Stop, target price Rs 520: HDFC ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 65.46 per cent stake in the com...,Sell,HDFC Securities,520.0,Shoppers Stop
3,"Buy Kotak Mahindra Bank, target price Rs 1936 ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 25.95 per cent stake in the com...,Buy,ICICI Direct,1936.0,Kotak Mahindra Bank
4,"Buy Mahindra & Mahindra, target price Rs 1243:...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Price on verge of breakout after consolidating...,Buy,ICICI Direct,1243.0,Mahindra & Mahindra
5,"Buy Infosys, target price Rs 1600: ICICI Direct",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Infosys key Products/Revenue Segments include ...,Buy,ICICI Direct,1600.0,Infosys
6,"Buy Motilal Oswal Financial Services, target p...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Motilal Oswal Financial is a diversified finan...,Buy,ICICI Direct,790.0,Motilal Oswal Financial Services
7,"Buy Star Health and Allied Insurance Company, ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 58.28 per cent stake in the com...,Buy,ICICI Direct,650.0,Star Health and Allied Insurance Company
8,"Buy SBI Life Insurance Company, target price R...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,"SBI Life Insurance Company, incorporated in th...",Buy,ICICI Direct,1350.0,SBI Life Insurance Company
9,"Buy Coforge, target price Rs 4725: ICICI Direct",https://economictimes.indiatimes.com/markets/s...,2023-04-28,"Coforge, incorporated in the year 1992, is a M...",Buy,ICICI Direct,4725.0,Coforge


In [16]:
df["time"] 

0      Apr 28, 2023, 01:52 PM IST
1      Apr 28, 2023, 01:44 PM IST
2      Apr 28, 2023, 01:33 PM IST
3      Apr 28, 2023, 12:05 PM IST
4      Apr 28, 2023, 11:30 AM IST
                  ...            
843    Feb 14, 2023, 09:41 AM IST
844    Feb 14, 2023, 09:35 AM IST
845    Feb 14, 2023, 09:28 AM IST
846    Feb 14, 2023, 09:23 AM IST
847    Feb 14, 2023, 09:17 AM IST
Name: time, Length: 848, dtype: object

In [36]:
format = "%b %d, %Y, %I:%M %p %Z"
date = datetime.datetime.strptime("Apr 28, 2023, 01:52 PM IST", format)
today = str(datetime.date.today())


In [37]:
date.strftime("%Y-%m-%d") == today

True

In [38]:
def convert_date(date: str):
    format = "%b %d, %Y, %I:%M %p %Z"
    date = datetime.datetime.strptime(date, format)
    return date.strftime("%Y-%m-%d")

In [40]:
df["date"] = df["time"].apply(convert_date)

In [41]:
df.head()

,title,link,time,text
0,"Buy Voltas, target price Rs 1005: BNP Paribas ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 30.3 per cent stake in the comp...
1,"Buy UTI Asset Management Company, target price...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,The brokerage expects the company to deliver 6...
2,"Sell Shoppers Stop, target price Rs 520: HDFC ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 65.46 per cent stake in the com...
3,"Buy Kotak Mahindra Bank, target price Rs 1936 ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 25.95 per cent stake in the com...
4,"Buy Mahindra & Mahindra, target price Rs 1243:...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Price on verge of breakout after consolidating...


In [58]:
title  = df["title"].iloc[0]
title

'Buy Voltas, target price Rs 1005: BNP Paribas Securities'

In [80]:
def get_target_price(title: str):
    target_price = "".join(re.findall(r"\d+", title))
    try:
        target_price = int(target_price)
    except:
        target_price = None
    return target_price

In [81]:
get_target_price(title)

1005

In [82]:
re.findall(r"\d+", title)

['1005']

In [91]:
df["target_price"] = df["title"].apply(get_target_price)
df["buy/sell"] = df["title"].apply(lambda x: x.split(" ")[0])
df["stock"] = df["title"].apply(lambda x: x.split(",")[0][4:])
df["fund"] = df["title"].apply(lambda x: x.split(":")[-1])
# df["date"] = df["time"].apply(convert_date)

ValueError: time data '2023-04-28' does not match format '%b %d, %Y, %I:%M %p %Z'

In [92]:
df.head()

,title,link,time,text,buy/sell,fund,target_price,stock
0,"Buy Voltas, target price Rs 1005: BNP Paribas ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 30.3 per cent stake in the comp...,Buy,BNP Paribas Securities,1005.0,Voltas
1,"Buy UTI Asset Management Company, target price...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,The brokerage expects the company to deliver 6...,Buy,HDFC Securities,770.0,UTI Asset Management Company
2,"Sell Shoppers Stop, target price Rs 520: HDFC ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 65.46 per cent stake in the com...,Sell,HDFC Securities,520.0,Shoppers Stop
3,"Buy Kotak Mahindra Bank, target price Rs 1936 ...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Promoters held 25.95 per cent stake in the com...,Buy,ICICI Direct,1936.0,Kotak Mahindra Bank
4,"Buy Mahindra & Mahindra, target price Rs 1243:...",https://economictimes.indiatimes.com/markets/s...,2023-04-28,Price on verge of breakout after consolidating...,Buy,ICICI Direct,1243.0,Mahindra & Mahindra
